In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
import time
import datetime
import json

import pinecone
import openai

openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

embedding_dimensions = 1536
model_engine = "text-embedding-ada-002"
pinecone_name = "dnd-rules-lawyer"
pinecone_region = "asia-southeast1-gcp" # Pinecone calls this the environement ? strange

pinecone.init(api_key=pinecone_api_key, environment=pinecone_region)
index = pinecone.Index(pinecone_name)
openai.api_key = openai_api_key

In [ ]:
def create_metadata(content, user, vector):
    create_time = time.time()
    return {
        "Timestamp": create_time,
        "User": "User",
        "Message": content,
        "Datetime": datetime.datetime.fromtimestamp(create_time),
        "Vector": vector
    }

In [ ]:
def create_message_hash(content):
    return hash(content)

In [ ]:
def get_openai_embeddings(content, engine="text-embedding-ada-002"):
    content = content.encode(encoding="ASCII", errors="ignore").decode()  # fix unicode errors
    response = openai.Embedding.create(input=content, engine=engine)
    vector = response['data'][0]['embedding']
    return vector

In [ ]:
def openai_completion(prompt, model="gpt-3.5-turbo", temperature=0, top_p=1.0, max_tokens=400, freq_pen=0.0, pres_pen=0.0, stops=["USER:"], max_retry=5):
        retry = 0
        prompt = prompt.encode(encodding="ASCII", errors="ignore").decode()
        while True:
            try:
                response = openai.Completion.create(
                    model=model,
                    prompt=prompt,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=top_p,
                    frequency_penalty=freq_pen,
                    presence_penalty=pres_pen,
                    stops=stops
                )
                text = response["choices"][0]["text"].strip()
                text = re.sub(r'[\r\n]+', "\n", text)
                text = re.sub(r'[\t]+', " ", text)
                filename = f'{time.time()}_gpt35_turbo.txt'
                if not os.path.exists('gpt3_logs'):
                    os.makedirs("gpt3_logs")
                    
            except Exception as e:
                retry += 1
                if retry >= max_retry:
                    return f"GPT3 error: {e}"
                print("Error in communication with openai.")
                time.sleep(1)

In [ ]:
def load_conversation(results):
    result = list()
    for m in results["matches"]:
        info = load_json(f'conversations/{m["id"]}.json')
        result.append(info)
    ordered = sorted(result, key=lambda d: d['time'], reverse=False)
    messages = [i["message"] for i in ordered]
    return "\n".join(messages).strip()

In [ ]:
def get_prompt(file_name):
    with open(file_name) as open_file:
        return open_file.read()

In [ ]:
def save_json(file_name, content):
    with open(file_name, "w+") as f:
        json.dump(content, f)

In [ ]:
get_prompt("prompt.txt")

In [ ]:
user = "USER"
top_k = 15
prompt_file = "prompt.txt"
logs_path = "./logs"


while True:
    payload = list()
    
    # Message Meta
    message = input("\n\n USER: ")
    message_hash = create_message_hash(message)

    # Create embedding of new message
    message_vector = get_openai_embeddings(message)
    
    # Save metadata about vector
    metadata = create_metadata(message, user, vector)
    save_json(logs_path, metadata)
    
    # Append to payload for later indexing
    # Send to Pinecone after gpt message
    payload.append((message_hash, message_vector))
    
    # Query for relevant messages, generate response
    results = index.query(vector=vector, top_k=top_k)
    conversation = load_conversation(results)
    prompt = get_prompt(prompt_file).replace("<PREVIOUS_CONVERSATION>")
    
    # Generate the response from the large lang model
    output = gpt3_completion(prompt)
    output_hash = (output)
    
    # Embed the output
    output_vector = get_openai_embeddings(output)
    
    # Save the output metadata
    metadata = create_metadata(message, user, vector)
    save_json(logs_path, metadata)
    
    # Append to the payload the response from gpt
    payload.append((output_hash, output_vector))
    
    # Upsert to the pinecone database
    index.upsert(payload)
    
    # Print responce to the message
    print(f'\n\n {output}')
    